# Pinterest Auto-uploader

This notebook automates uploading images to Pinterest using Selenium and Chrome WebDriver in Google Colab.

## Setup
First, we need to install the required packages and Chrome WebDriver.

In [1]:
# Install required packages
!pip install selenium
!apt-get update
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# Add ChromeDriver to path
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Check installation
!which chromedriver
!chromedriver --version

print("\nSetup complete! All required packages installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 10.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,926 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages 

First version

In [2]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from google.colab import files
import time
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Upload a wallpaper image
print("Please upload a wallpaper image:")
uploaded = files.upload()

# Save the uploaded image
image_filename = next(iter(uploaded))
image_path = f"/content/{image_filename}"
print(f"Saved uploaded image to {image_path}")

def upload_to_pinterest(email, password, image_path, title, description, board_name):
    """
    Upload an image to Pinterest using Selenium in Google Colab
    """
    # Configure Chrome options specifically for Colab
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36')

    try:
        # Initialize the WebDriver
        logger.info("Initializing Chrome WebDriver...")
        driver = webdriver.Chrome(options=chrome_options)

        # Make the driver window size larger
        driver.set_window_size(1920, 1080)

        # Set a longer wait time for Colab
        wait = WebDriverWait(driver, 30)

        # Step 1: Login to Pinterest
        logger.info("Navigating to Pinterest login page...")
        driver.get("https://www.pinterest.com/login/")

        # Take a screenshot to help debug
        driver.save_screenshot("login_page.png")
        files.download("login_page.png")

        # Wait for the login page to fully load
        time.sleep(5)

        # Wait for the login page to load and enter email
        logger.info("Entering email...")
        email_field = wait.until(EC.element_to_be_clickable((By.ID, "email")))
        email_field.clear()
        email_field.send_keys(email)

        # Enter password
        logger.info("Entering password...")
        password_field = driver.find_element(By.ID, "password")
        password_field.clear()
        password_field.send_keys(password)

        # Take screenshot before clicking login
        driver.save_screenshot("before_login.png")
        files.download("before_login.png")

        # Click login button
        logger.info("Clicking login button...")
        login_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
        login_button.click()

        # Wait longer for login to complete
        time.sleep(10)

        # Take screenshot after login
        driver.save_screenshot("after_login.png")
        files.download("after_login.png")

        # Navigate to pin creation page
        logger.info("Navigating to pin creation page...")
        driver.get("https://www.pinterest.com/pin-builder/")

        time.sleep(5)
        driver.save_screenshot("pin_builder.png")
        files.download("pin_builder.png")

        # Find the file input directly - using the exact ID from the HTML provided
        logger.info("Looking for file input element...")
        time.sleep(5)

        # Take a screenshot to see the page state
        driver.save_screenshot("before_file_upload.png")
        files.download("before_file_upload.png")

        # Using the exact ID from the HTML provided
        file_input_id = "storyboard-upload-input"

        try:
            # First try using the ID
            file_input = driver.find_element(By.ID, file_input_id)
            logger.info(f"Found file input by ID: {file_input_id}")
        except:
            # If the ID fails, fall back to a more generic selector
            file_input = driver.find_element(By.CSS_SELECTOR, "input[type='file'][accept*='image']")
            logger.info("Found file input using generic selector")

        # Make sure the file input is visible and interactable
        driver.execute_script("arguments[0].style.opacity = '1'; arguments[0].style.display = 'block'; arguments[0].style.visibility = 'visible';", file_input)

        # Send the absolute file path to the input
        logger.info(f"Uploading image: {os.path.abspath(image_path)}")
        file_input.send_keys(os.path.abspath(image_path))

        logger.info("Waiting for image to upload...")
        time.sleep(10)  # Give it time to upload

        # Take a screenshot after upload
        driver.save_screenshot("after_upload.png")
        files.download("after_upload.png")

        # Add title using a combined approach based on the GitHub implementation
        logger.info("Entering pin title with GitHub implementation approach...")

        # Wait longer for page to fully load after image upload
        logger.info("Waiting for page elements to stabilize...")
        time.sleep(15)

        # Take screenshot before trying to add title
        driver.save_screenshot("before_title_input.png")
        files.download("before_title_input.png")

        # Implement title entry using XPath approach similar to GitHub code
        title_set = False

        try:
            # Approach 1: Try using the textarea with pin-draft-title ID (from GitHub code)
            logger.info("Trying to locate title field using pin-draft-title XPath...")
            title_xpath = "//textarea[contains(@id, 'pin-draft-title')]"

            # Wait for the element to be visible
            title_input = wait.until(EC.visibility_of_element_located((By.XPATH, title_xpath)))

            # Clear and enter text
            title_input.clear()
            title_input.send_keys(title)
            logger.info(f"Successfully entered title '{title}' using XPath approach")
            title_set = True

            # Take screenshot after successful entry
            driver.save_screenshot("after_title_xpath.png")
            files.download("after_title_xpath.png")

        except Exception as e:
            logger.warning(f"GitHub XPath method failed: {str(e)}")

            # Approach 2: Try using JavaScript for the textarea approach
            try:
                logger.info("Trying JavaScript with textarea selector...")
                js_result = driver.execute_script("""
                    // Try to find the title input using various selectors based on GitHub implementation
                    const titleInput = document.querySelector('textarea[id*="pin-draft-title"]');

                    if (titleInput) {
                        // Make sure it's visible
                        titleInput.style.display = 'block';
                        titleInput.style.visibility = 'visible';

                        // Clear and set value
                        titleInput.value = '';
                        titleInput.value = arguments[0];

                        // Trigger events
                        titleInput.dispatchEvent(new Event('input', {bubbles: true}));
                        titleInput.dispatchEvent(new Event('change', {bubbles: true}));

                        return 'Title set with textarea selector: ' + titleInput.value;
                    }
                    return null;
                """, title)

                if js_result:
                    logger.info(f"JavaScript textarea result: {js_result}")
                    title_set = True
                else:
                    logger.warning("Textarea title approach failed")

            except Exception as e:
                logger.warning(f"JavaScript textarea approach failed: {str(e)}")

            # Approach 3: Try using DOM manipulation with storyboard-selector-title (previous working approach)
            if not title_set:
                try:
                    # Fall back to the previous approach with storyboard-selector-title
                    js_result = driver.execute_script("""
                    // Function to attempt finding and setting the title with delay
                    function setTitleWithRetry(title, maxRetries=5, delayMs=1000) {
                        let attempt = 0;

                        function attemptSetTitle() {
                            attempt++;
                            console.log(`Title setting attempt ${attempt}`);

                            // Strategy 1: Find by exact HTML structure provided
                            let container = document.querySelector('.XiG.zI7.iyn.Hsu');
                            if (container) {
                                let input = container.querySelector('input');
                                if (input) {
                                    trySetValue(input, title);
                                    return true;
                                }
                            }

                            // Strategy 2: Find by ID
                            let titleInput = document.getElementById('storyboard-selector-title');
                            if (titleInput) {
                                trySetValue(titleInput, title);
                                return true;
                            }

                            // Strategy 3: Find by placeholder text
                            let placeholderInput = document.querySelector('input[placeholder="Add a title"]');
                            if (placeholderInput) {
                                trySetValue(placeholderInput, title);
                                return true;
                            }

                            // Strategy 4: Find any input in the storyboard container
                            let storyboardContainer = document.querySelector('[data-test-id="storyboard-title-field-container"]');
                            if (storyboardContainer) {
                                let input = storyboardContainer.querySelector('input');
                                if (input) {
                                    trySetValue(input, title);
                                    return true;
                                }
                            }

                            // If we still haven't found it and have retries left
                            if (attempt < maxRetries) {
                                console.log(`Title input not found, retrying in ${delayMs}ms...`);
                                setTimeout(attemptSetTitle, delayMs);
                                return false;
                            }

                            return false;
                        }

                        // Helper function to properly set a value on an input
                        function trySetValue(input, value) {
                            // Make sure the element is visible and active
                            input.style.display = 'block';
                            input.style.visibility = 'visible';
                            input.disabled = false;
                            input.scrollIntoView({block: 'center'});

                            // Focus and clear the input
                            input.focus();
                            input.value = '';

                            // Set the value
                            input.value = value;

                            // Dispatch multiple events to ensure it's recognized
                            input.dispatchEvent(new Event('focus'));
                            input.dispatchEvent(new Event('input', {bubbles: true}));
                            input.dispatchEvent(new Event('change', {bubbles: true}));

                            console.log('Title set to:', value);
                        }

                        // Start the attempt sequence
                        return attemptSetTitle();
                    }

                    // Execute with the title from arguments
                    return setTitleWithRetry(arguments[0]);
                """, title)

                    logger.info(f"JavaScript title setting result: {js_result}")
                    title_set = True

                except Exception as e:
                    logger.error(f"All standard title entry methods failed: {str(e)}")

        # Approach 4: Try the role='combobox' method from GitHub code as last resort
        if not title_set:
            try:
                logger.info("Trying role='combobox' approach for title...")

                # This is similar to the description approach in the GitHub code
                # Sometimes Pinterest uses similar fields for title and description
                combobox_xpath = "//div[@role='combobox']/div/div/div"
                combobox_element = wait.until(EC.visibility_of_element_located((By.XPATH, combobox_xpath)))

                # Click to activate
                combobox_element.click()

                # Enter title character by character with small delays
                for char in title:
                    combobox_element.send_keys(char)
                    time.sleep(0.05)

                logger.info("Entered title using combobox element")
                title_set = True

            except Exception as e:
                logger.error(f"Role='combobox' approach failed: {str(e)}")

        # Take a screenshot of the final state
        driver.save_screenshot("final_title_state.png")
        files.download("final_title_state.png")

        if not title_set:
            logger.warning(f"Could not set title to '{title}' after multiple attempts")

        # Enter description with improved method
        logger.info("Entering pin description...")
        try:
            # Wait for the description area to be clickable
            description_area = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".public-DraftEditor-content")))
            description_area.click()

            # Type the description with small pauses to make it more reliable
            for char in description:
                description_area.send_keys(char)
                time.sleep(0.05)

            logger.info("Description entered successfully")
        except Exception as e:
            logger.error(f"Error entering description: {str(e)}")
            # Fall back to JavaScript approach
            try:
                driver.execute_script("""
                    // Look for the Draft.js editor content area
                    var editorDiv = document.querySelector('.public-DraftEditor-content');
                    if (editorDiv) {
                        // Focus the editor
                        editorDiv.focus();

                        // Set content with an input event
                        editorDiv.innerHTML = '<div data-contents="true"><div data-block="true" data-editor="13keg" data-offset-key="0-0-0"><div data-offset-key="0-0-0" class="public-DraftStyleDefault-block public-DraftStyleDefault-ltr"><span data-offset-key="0-0-0"><span data-text="true">' + arguments[0] + '</span></span></div></div></div>';

                        // Trigger input event
                        var event = new Event('input', { bubbles: true });
                        editorDiv.dispatchEvent(event);
                    }
                """, description)
                logger.info("Description entered via JavaScript to Draft.js editor")
            except Exception as e:
                logger.error(f"Error entering description via JavaScript: {str(e)}")

        # Select the board - using improved approach with data-test-id
        try:
            logger.info("Selecting board...")
            # Take a screenshot before board selection
            driver.save_screenshot("before_board_selection.png")
            files.download("before_board_selection.png")

            # Find and click the board dropdown
            board_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-test-id='board-dropdown-select-button']")))
            board_dropdown.click()
            time.sleep(3)  # Wait for dropdown to fully appear

            # Take a screenshot of the dropdown
            driver.save_screenshot("board_dropdown_open.png")
            files.download("board_dropdown_open.png")

            # Select the specific board using the data-test-id structure
            specific_board = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"[data-test-id='board-row-{board_name}']")))
            logger.info(f"Found specific board element: {board_name}")
            specific_board.click()

            logger.info(f"Selected board: {board_name}")
            time.sleep(2)
        except Exception as e:
            logger.error(f"Error selecting board with data-test-id: {str(e)}")
            # Fall back to the previous methods
            try:
                # Look for board by name in dropdown items
                board_options = driver.find_elements(By.CSS_SELECTOR, "div[role='option'] div.X8m")
                for option in board_options:
                    option_text = option.text.strip()
                    logger.info(f"Found board option: {option_text}")
                    if board_name.lower() in option_text.lower():
                        # Click the parent option element, not just the text
                        parent = driver.execute_script("return arguments[0].closest('[role=\"option\"]')", option)
                        if parent:
                            parent.click()
                            logger.info(f"Selected board: {option_text}")
                            time.sleep(2)
                            break
            except Exception as e:
                logger.error(f"Error selecting board by option text: {str(e)}")
                # Try JavaScript approach as last resort
                try:
                    driver.execute_script("""
                        // Find and click the board dropdown
                        var boardDropdown = document.querySelector('[data-test-id="board-dropdown-select-button"]');
                        if (boardDropdown) {
                            boardDropdown.click();

                            // Wait for dropdown to appear
                            setTimeout(function() {
                                // Find the board option by name
                                var boardName = arguments[0];
                                var options = Array.from(document.querySelectorAll('div[role="option"]'));

                                // Look for the board by name
                                for (var i = 0; i < options.length; i++) {
                                    if (options[i].innerText.toLowerCase().includes(boardName.toLowerCase())) {
                                        options[i].click();
                                        console.log("Selected board via JavaScript: " + options[i].innerText);
                                        break;
                                    }
                                }
                            }, 1000);
                        }
                    """, board_name)
                    logger.info("Attempted to select board via JavaScript")
                except Exception as e:
                    logger.error(f"Error selecting board via JavaScript: {str(e)}")

        # Take a screenshot before publishing
        driver.save_screenshot("before_publish.png")
        files.download("before_publish.png")

        # Find and click the Publish button
        logger.info("Publishing pin...")
        try:
            # Based on the screenshot, look for the red Publish button
            publish_button = driver.find_element(By.CSS_SELECTOR, "button.RCK.Hsu.USg.adn.gn8.L4E.kVc.CCY.oRi.lnZ.wsz")
            publish_button_text = publish_button.text.strip()
            logger.info(f"Found publish button: {publish_button_text}")
            if "publish" in publish_button_text.lower():
                publish_button.click()
                logger.info("Clicked publish button")
            else:
                # Try to find by text content
                buttons = driver.find_elements(By.TAG_NAME, "button")
                for button in buttons:
                    if "publish" in button.text.lower():
                        button.click()
                        logger.info(f"Clicked button with text: {button.text}")
                        break
        except Exception as e:
            logger.error(f"Error finding publish button: {str(e)}")
            # Try JavaScript approach
            try:
                driver.execute_script("""
                    // Find a button with "Publish" text
                    var buttons = Array.from(document.querySelectorAll('button'));
                    for (var i = 0; i < buttons.length; i++) {
                        if (buttons[i].innerText.toLowerCase().includes('publish')) {
                            console.log("Found publish button via JavaScript: " + buttons[i].innerText);
                            buttons[i].click();
                            return true;
                        }
                    }

                    // If no "Publish" button found, look for one with a distinctive appearance (e.g., red)
                    var redButtons = Array.from(document.querySelectorAll('button')).filter(function(btn) {
                        var style = window.getComputedStyle(btn);
                        var bgColor = style.backgroundColor.toLowerCase();
                        return bgColor.includes('rgb(230, 0, 35)') || bgColor.includes('#e60023');
                    });

                    if (redButtons.length > 0) {
                        console.log("Found red button, clicking it");
                        redButtons[0].click();
                        return true;
                    }

                    return false;
                """)
                logger.info("Attempted to click publish button via JavaScript")
            except Exception as e:
                logger.error(f"Error clicking publish button via JavaScript: {str(e)}")

        # Wait for the pin to be published
        time.sleep(15)

        # Check if we've been redirected (sign of success)
        current_url = driver.current_url
        logger.info(f"Current URL after publish attempt: {current_url}")

        # Take a final screenshot
        driver.save_screenshot("pin_published.png")
        files.download("pin_published.png")

        # Determine if publish was successful
        success = False
        if "pin-builder" not in current_url:
            success = True
            logger.info("Pin appears to have been published successfully (redirected from pin-builder)")
        else:
            # Check for success messages on the page
            success_text = driver.execute_script("""
                return document.body.innerText.includes('Saved to') ||
                       document.body.innerText.includes('Pin created') ||
                       document.body.innerText.includes('Your Pin was saved') ||
                       document.body.innerText.includes('Successfully published');
            """)
            if success_text:
                success = True
                logger.info("Pin appears to have been published successfully (success message found)")

        if success:
            logger.info("Pin published successfully!")
            return True
        else:
            logger.warning("Pin may not have been published successfully")
            return False

    except Exception as e:
        logger.error(f"Error during Pinterest upload: {str(e)}")
        # Include screenshot of the error state
        try:
            driver.save_screenshot("error_state.png")
            files.download("error_state.png")
        except:
            pass
        return False

    finally:
        # Close the browser
        logger.info("Closing browser...")
        try:
            driver.quit()
        except:
            pass

# Get user inputs
EMAIL = "mojomaniac2005@gmail.com"
PASSWORD = "Mojo@2005"
TITLE = "Sample Wallpaper Title"
DESCRIPTION = "Sample Wallpaper"
BOARD_NAME = "Wallpapers"

# Upload the wallpaper
result = upload_to_pinterest(EMAIL, PASSWORD, image_path, TITLE, DESCRIPTION, BOARD_NAME)

if result:
    print("Wallpaper uploaded successfully!")
else:
    print("Failed to upload wallpaper, check the logs for details.")

Please upload a wallpaper image:


Saving my-notion-face-transparent (1).png to my-notion-face-transparent (1).png
Saved uploaded image to /content/my-notion-face-transparent (1).png


KeyboardInterrupt: 

New version here

In [ ]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from google.colab import files
import time
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Upload a wallpaper image
print("Please upload a wallpaper image:")
uploaded = files.upload()

# Save the uploaded image
image_filename = next(iter(uploaded))
image_path = f"/content/{image_filename}"
print(f"Saved uploaded image to {image_path}")

def upload_to_pinterest(email, password, image_path, title, description, board_name, link_url):
    """
    Upload an image to Pinterest using Selenium in Google Colab
    """
    # Configure Chrome options specifically for Colab
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36')

    try:
        # Initialize the WebDriver
        logger.info("Initializing Chrome WebDriver...")
        driver = webdriver.Chrome(options=chrome_options)

        # Make the driver window size larger
        driver.set_window_size(1920, 1080)

        # Set a longer wait time for Colab
        wait = WebDriverWait(driver, 30)

        # Step 1: Login to Pinterest
        logger.info("Navigating to Pinterest login page...")
        driver.get("https://www.pinterest.com/login/")

        # Take a screenshot to help debug
        driver.save_screenshot("login_page.png")
        files.download("login_page.png")

        # Wait for the login page to fully load
        time.sleep(5)

        # Wait for the login page to load and enter email
        logger.info("Entering email...")
        email_field = wait.until(EC.element_to_be_clickable((By.ID, "email")))
        email_field.clear()
        email_field.send_keys(email)

        # Enter password
        logger.info("Entering password...")
        password_field = driver.find_element(By.ID, "password")
        password_field.clear()
        password_field.send_keys(password)

        # Take screenshot before clicking login
        driver.save_screenshot("before_login.png")
        files.download("before_login.png")

        # Click login button
        logger.info("Clicking login button...")
        login_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
        login_button.click()

        # Wait longer for login to complete
        time.sleep(10)

        # Take screenshot after login
        driver.save_screenshot("after_login.png")
        files.download("after_login.png")

        # Navigate to pin creation page
        logger.info("Navigating to pin creation page...")
        driver.get("https://www.pinterest.com/pin-builder/")

        time.sleep(5)
        driver.save_screenshot("pin_builder.png")
        files.download("pin_builder.png")

        # Find the file input directly - using the exact ID from the HTML provided
        logger.info("Looking for file input element...")
        time.sleep(5)

        # Take a screenshot to see the page state
        driver.save_screenshot("before_file_upload.png")
        files.download("before_file_upload.png")

        # Using the exact ID from the HTML provided
        file_input_id = "storyboard-upload-input"

        try:
            # First try using the ID
            file_input = driver.find_element(By.ID, file_input_id)
            logger.info(f"Found file input by ID: {file_input_id}")
        except:
            # If the ID fails, fall back to a more generic selector
            file_input = driver.find_element(By.CSS_SELECTOR, "input[type='file'][accept*='image']")
            logger.info("Found file input using generic selector")

        # Make sure the file input is visible and interactable
        driver.execute_script("arguments[0].style.opacity = '1'; arguments[0].style.display = 'block'; arguments[0].style.visibility = 'visible';", file_input)

        # Send the absolute file path to the input
        logger.info(f"Uploading image: {os.path.abspath(image_path)}")
        file_input.send_keys(os.path.abspath(image_path))

        logger.info("Waiting for image to upload...")
        time.sleep(10)  # Give it time to upload

        # Take a screenshot after upload
        driver.save_screenshot("after_upload.png")
        files.download("after_upload.png")

        # Add title using a combined approach based on the GitHub implementation
        logger.info("Entering pin title with GitHub implementation approach...")

        # Wait longer for page to fully load after image upload
        logger.info("Waiting for page elements to stabilize...")
        time.sleep(15)

        # Take screenshot before trying to add title
        driver.save_screenshot("before_title_input.png")
        files.download("before_title_input.png")

        # Implement title entry using XPath approach similar to GitHub code
        title_set = False

        try:
            # Approach 1: Try using the textarea with pin-draft-title ID (from GitHub code)
            logger.info("Trying to locate title field using pin-draft-title XPath...")
            title_xpath = "//textarea[contains(@id, 'pin-draft-title')]"

            # Wait for the element to be visible
            title_input = wait.until(EC.visibility_of_element_located((By.XPATH, title_xpath)))

            # Clear and enter text
            title_input.clear()
            title_input.send_keys(title)
            logger.info(f"Successfully entered title '{title}' using XPath approach")
            title_set = True

            # Take screenshot after successful entry
            driver.save_screenshot("after_title_xpath.png")
            files.download("after_title_xpath.png")

        except Exception as e:
            logger.warning(f"GitHub XPath method failed: {str(e)}")

            # Approach 2: Try using JavaScript for the textarea approach
            try:
                logger.info("Trying JavaScript with textarea selector...")
                js_result = driver.execute_script("""
                    // Try to find the title input using various selectors based on GitHub implementation
                    const titleInput = document.querySelector('textarea[id*="pin-draft-title"]');

                    if (titleInput) {
                        // Make sure it's visible
                        titleInput.style.display = 'block';
                        titleInput.style.visibility = 'visible';

                        // Clear and set value
                        titleInput.value = '';
                        titleInput.value = arguments[0];

                        // Trigger events
                        titleInput.dispatchEvent(new Event('input', {bubbles: true}));
                        titleInput.dispatchEvent(new Event('change', {bubbles: true}));

                        return 'Title set with textarea selector: ' + titleInput.value;
                    }
                    return null;
                """, title)

                if js_result:
                    logger.info(f"JavaScript textarea result: {js_result}")
                    title_set = True
                else:
                    logger.warning("Textarea title approach failed")

            except Exception as e:
                logger.warning(f"JavaScript textarea approach failed: {str(e)}")

            # Approach 3: Try using DOM manipulation with storyboard-selector-title (previous working approach)
            if not title_set:
                try:
                    # Fall back to the previous approach with storyboard-selector-title
                    js_result = driver.execute_script("""
                    // Function to attempt finding and setting the title with delay
                    function setTitleWithRetry(title, maxRetries=5, delayMs=1000) {
                        let attempt = 0;

                        function attemptSetTitle() {
                            attempt++;
                            console.log(`Title setting attempt ${attempt}`);

                            // Strategy 1: Find by exact HTML structure provided
                            let container = document.querySelector('.XiG.zI7.iyn.Hsu');
                            if (container) {
                                let input = container.querySelector('input');
                                if (input) {
                                    trySetValue(input, title);
                                    return true;
                                }
                            }

                            // Strategy 2: Find by ID
                            let titleInput = document.getElementById('storyboard-selector-title');
                            if (titleInput) {
                                trySetValue(titleInput, title);
                                return true;
                            }

                            // Strategy 3: Find by placeholder text
                            let placeholderInput = document.querySelector('input[placeholder="Add a title"]');
                            if (placeholderInput) {
                                trySetValue(placeholderInput, title);
                                return true;
                            }

                            // Strategy 4: Find any input in the storyboard container
                            let storyboardContainer = document.querySelector('[data-test-id="storyboard-title-field-container"]');
                            if (storyboardContainer) {
                                let input = storyboardContainer.querySelector('input');
                                if (input) {
                                    trySetValue(input, title);
                                    return true;
                                }
                            }

                            // If we still haven't found it and have retries left
                            if (attempt < maxRetries) {
                                console.log(`Title input not found, retrying in ${delayMs}ms...`);
                                setTimeout(attemptSetTitle, delayMs);
                                return false;
                            }

                            return false;
                        }

                        // Helper function to properly set a value on an input
                        function trySetValue(input, value) {
                            // Make sure the element is visible and active
                            input.style.display = 'block';
                            input.style.visibility = 'visible';
                            input.disabled = false;
                            input.scrollIntoView({block: 'center'});

                            // Focus and clear the input
                            input.focus();
                            input.value = '';

                            // Set the value
                            input.value = value;

                            // Dispatch multiple events to ensure it's recognized
                            input.dispatchEvent(new Event('focus'));
                            input.dispatchEvent(new Event('input', {bubbles: true}));
                            input.dispatchEvent(new Event('change', {bubbles: true}));

                            console.log('Title set to:', value);
                        }

                        // Start the attempt sequence
                        return attemptSetTitle();
                    }

                    // Execute with the title from arguments
                    return setTitleWithRetry(arguments[0]);
                """, title)

                    logger.info(f"JavaScript title setting result: {js_result}")
                    title_set = True

                except Exception as e:
                    logger.error(f"All standard title entry methods failed: {str(e)}")

        # Approach 4: Try the role='combobox' method from GitHub code as last resort
        if not title_set:
            try:
                logger.info("Trying role='combobox' approach for title...")

                # This is similar to the description approach in the GitHub code
                # Sometimes Pinterest uses similar fields for title and description
                combobox_xpath = "//div[@role='combobox']/div/div/div"
                combobox_element = wait.until(EC.visibility_of_element_located((By.XPATH, combobox_xpath)))

                # Click to activate
                combobox_element.click()

                # Enter title character by character with small delays
                for char in title:
                    combobox_element.send_keys(char)
                    time.sleep(0.05)

                logger.info("Entered title using combobox element")
                title_set = True

            except Exception as e:
                logger.error(f"Role='combobox' approach failed: {str(e)}")

        # Take a screenshot of the final state
        driver.save_screenshot("final_title_state.png")
        files.download("final_title_state.png")

        if not title_set:
            logger.warning(f"Could not set title to '{title}' after multiple attempts")

        # Enter description with improved method
        logger.info("Entering pin description...")
        try:
            # Wait for the description area to be clickable
            description_area = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".public-DraftEditor-content")))
            description_area.click()

            # Type the description with small pauses to make it more reliable
            for char in description:
                description_area.send_keys(char)
                time.sleep(0.05)

            logger.info("Description entered successfully")
        except Exception as e:
            logger.error(f"Error entering description: {str(e)}")
            # Fall back to JavaScript approach
            try:
                driver.execute_script("""
                    // Look for the Draft.js editor content area
                    var editorDiv = document.querySelector('.public-DraftEditor-content');
                    if (editorDiv) {
                        // Focus the editor
                        editorDiv.focus();

                        // Set content with an input event
                        editorDiv.innerHTML = '<div data-contents="true"><div data-block="true" data-editor="13keg" data-offset-key="0-0-0"><div data-offset-key="0-0-0" class="public-DraftStyleDefault-block public-DraftStyleDefault-ltr"><span data-offset-key="0-0-0"><span data-text="true">' + arguments[0] + '</span></span></div></div></div>';

                        // Trigger input event
                        var event = new Event('input', { bubbles: true });
                        editorDiv.dispatchEvent(event);
                    }
                """, description)
                logger.info("Description entered via JavaScript to Draft.js editor")
            except Exception as e:
                logger.error(f"Error entering description via JavaScript: {str(e)}")

        # Add link URL - improved implementation with multiple approaches and retries
        logger.info(f"Adding link URL: {link_url}")
        # Take a screenshot before attempting to add link
        driver.save_screenshot("before_link_attempt.png")
        files.download("before_link_attempt.png")

        # Wait extra time before attempting to add link to ensure page is fully loaded
        logger.info("Waiting for page to stabilize before adding link...")
        time.sleep(10)

        link_added = False

        # Approach 1: Find by aria-label (commonly used by Pinterest)
        try:
            logger.info("Trying to find link field by aria-label...")
            link_input = wait.until(EC.element_to_be_clickable(
                (By.CSS_SELECTOR, "input[aria-label='Add a destination link']")))
            link_input.clear()
            link_input.send_keys(link_url)
            link_added = True
            logger.info("Link URL added successfully via aria-label selector")

            # Take a screenshot after adding link
            driver.save_screenshot("link_added_aria_label.png")
            files.download("link_added_aria_label.png")
        except Exception as e:
            logger.warning(f"Could not find link field by aria-label: {str(e)}")

        # Approach 2: Find by placeholder
        if not link_added:
            try:
                logger.info("Trying to find link field by placeholder...")
                # Pinterest often uses 'Add a destination link' or 'Add a link' as placeholder
                link_input = wait.until(EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "input[placeholder*='link' i]")))
                link_input.clear()
                link_input.send_keys(link_url)
                link_added = True
                logger.info("Link URL added successfully via placeholder selector")

                # Take a screenshot after adding link
                driver.save_screenshot("link_added_placeholder.png")
                files.download("link_added_placeholder.png")
            except Exception as e:
                logger.warning(f"Could not find link field by placeholder: {str(e)}")

        # Approach 3: Search for any visible input with 'link' in label
        if not link_added:
            try:
                logger.info("Looking for any input with 'link' in its label...")
                # Capture all inputs and their associated text
                inputs_with_labels = driver.execute_script("""
                    function getInputsWithLabels() {
                        const inputs = document.querySelectorAll('input[type="text"], input[type="url"], input:not([type])');
                        const results = [];

                        inputs.forEach(input => {
                            // Check if input is visible
                            if (input.offsetParent !== null) {
                                // Check the input itself
                                const inputObj = {
                                    element: input,
                                    id: input.id,
                                    placeholder: input.placeholder,
                                    ariaLabel: input.getAttribute('aria-label'),
                                    nearbyText: ''
                                };

                                // Look for nearby labels or text
                                const parent = input.parentElement;
                                if (parent) {
                                    inputObj.nearbyText = parent.innerText;
                                }

                                results.push(inputObj);
                            }
                        });
                        return results;
                    }
                    return getInputsWithLabels();
                """)

                # Find an input that might be for links
                link_field_found = False
                for input_data in inputs_with_labels:
                    input_info = driver.execute_script("return arguments[0]", input_data)
                    logger.info(f"Found input: ID={input_info['id']}, Placeholder={input_info['placeholder']}, AriaLabel={input_info['ariaLabel']}")

                    # Check if any of the text suggests this is for links
                    if ('link' in (input_info['id'] or '').lower() or
                        'link' in (input_info['placeholder'] or '').lower() or
                        'link' in (input_info['ariaLabel'] or '').lower() or
                        'link' in (input_info['nearbyText'] or '').lower() or
                        'url' in (input_info['id'] or '').lower() or
                        'website' in (input_info['id'] or '').lower()):

                        # Found a likely candidate
                        element = input_data['element']
                        driver.execute_script("arguments[0].scrollIntoView({block: 'center'}); arguments[0].focus();", element)
                        driver.execute_script("arguments[0].value = ''; arguments[0].value = arguments[1]; arguments[0].dispatchEvent(new Event('input', {bubbles: true}));", element, link_url)
                        logger.info("Set link value via JavaScript DOM manipulation")
                        link_field_found = True
                        break

                if link_field_found:
                    link_added = True
                    # Take a screenshot after adding link
                    driver.save_screenshot("link_added_js_search.png")
                    files.download("link_added_js_search.png")
                else:
                    logger.warning("Could not identify a suitable link input field from DOM analysis")
            except Exception as e:
                logger.warning(f"Error in searching for link field: {str(e)}")

        # Approach 4: Look for the link section and try to click it first
        if not link_added:
            try:
                logger.info("Trying to find and click link section first...")
                # Try to find a container or section related to links
                link_sections = driver.find_elements(By.XPATH, "//div[contains(text(), 'Link') or contains(text(), 'Website') or contains(text(), 'URL')]/following::input[1]")

                if link_sections:
                    logger.info(f"Found {len(link_sections)} potential link sections")
                    link_section = link_sections[0]
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link_section)
                    link_section.click()
                    time.sleep(1)  # Wait for UI to respond
                    link_section.clear()
                    link_section.send_keys(link_url)
                    link_added = True
                    logger.info("Added link via section finder method")

                    # Take a screenshot
                    driver.save_screenshot("link_added_section_finder.png")
                    files.download("link_added_section_finder.png")
            except Exception as e:
                logger.warning(f"Error finding link section: {str(e)}")

        # Approach 5: Most aggressive - inject link directly via JavaScript
        if not link_added:
            try:
                logger.info("Attempting to inject link directly via JavaScript...")
                result = driver.execute_script("""
                    // Try every possible way to find and set the link field
                    function setLinkWithMultipleApproaches(linkUrl) {
                        // Store if we succeeded
                        let linkSet = false;

                        // Approach 1: Try common IDs
                        const commonLinkIds = ['WebsiteField', 'link-field', 'url-field', 'website-url', 'link_url', 'site_url'];
                        for (const id of commonLinkIds) {
                            const field = document.getElementById(id);
                            if (field) {
                                field.value = linkUrl;
                                field.dispatchEvent(new Event('input', {bubbles: true}));
                                field.dispatchEvent(new Event('change', {bubbles: true}));
                                console.log('Set link via ID: ' + id);
                                linkSet = true;
                                break;
                            }
                        }

                        // Approach 2: Find any input with link/url related attributes
                        if (!linkSet) {
                            const allInputs = document.querySelectorAll('input');
                            for (const input of allInputs) {
                                const inputText = (input.id + ' ' + input.placeholder + ' ' +
                                                 (input.getAttribute('aria-label') || '') + ' ' +
                                                  input.name).toLowerCase();
                                if (inputText.includes('link') || inputText.includes('url') || inputText.includes('website')) {
                                    input.value = linkUrl;
                                    input.dispatchEvent(new Event('input', {bubbles: true}));
                                    input.dispatchEvent(new Event('change', {bubbles: true}));
                                    console.log('Set link via attribute search');
                                    linkSet = true;
                                    break;
                                }
                            }
                        }

                        // Approach 3: Create our own input if needed
                        if (!linkSet) {
                            // Look for link container
                            const possibleContainers = Array.from(document.querySelectorAll('div, section')).filter(el => {
                                return el.innerText.toLowerCase().includes('link') ||
                                       el.innerText.toLowerCase().includes('website') ||
                                       el.innerText.toLowerCase().includes('url');
                            });

                            if (possibleContainers.length > 0) {
                                const container = possibleContainers[0];

                                // Look for any inputs inside this container
                                const containerInputs = container.querySelectorAll('input');
                                if (containerInputs.length > 0) {
                                    containerInputs[0].value = linkUrl;
                                    containerInputs[0].dispatchEvent(new Event('input', {bubbles: true}));
                                    containerInputs[0].dispatchEvent(new Event('change', {bubbles: true}));
                                    console.log('Set link via container > input');
                                    linkSet = true;
                                }
                            }
                        }

                        // Approach 4: Look for data attributes specific to Pinterest
                        if (!linkSet) {
                            const pinterestElements = document.querySelectorAll('[data-test-id*="link"]');
                            for (const element of pinterestElements) {
                                const input = element.querySelector('input');
                                if (input) {
                                    input.value = linkUrl;
                                    input.dispatchEvent(new Event('input', {bubbles: true}));
                                    input.dispatchEvent(new Event('change', {bubbles: true}));
                                    console.log('Set link via Pinterest data-test-id');
                                    linkSet = true;
                                    break;
                                }
                            }
                        }

                        // Return whether we were successful
                        return {
                            success: linkSet,
                            message: linkSet ? 'Link set successfully via JavaScript' : 'Could not set link via JavaScript'
                        };
                    }

                    return setLinkWithMultipleApproaches(arguments[0]);
                """, link_url)

                if result['success']:
                    link_added = True
                    logger.info(f"Link added via JavaScript: {result['message']}")
                    # Take a screenshot
                    driver.save_screenshot("link_added_js_direct.png")
                    files.download("link_added_js_direct.png")
                else:
                    logger.warning(f"JavaScript link injection result: {result['message']}")
            except Exception as e:
                logger.error(f"Error in JavaScript link injection: {str(e)}")

        if not link_added:
            logger.warning("COULD NOT ADD LINK URL after multiple attempts. Continuing with upload anyway.")
        else:
            logger.info("Successfully added link URL to pin")

        # Take a final screenshot after link attempts
        driver.save_screenshot("after_all_link_attempts.png")
        files.download("after_all_link_attempts.png")

        # Select the board - using improved approach with data-test-id
        try:
            logger.info("Selecting board...")
            # Take a screenshot before board selection
            driver.save_screenshot("before_board_selection.png")
            files.download("before_board_selection.png")

            # Find and click the board dropdown
            board_dropdown = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-test-id='board-dropdown-select-button']")))
            board_dropdown.click()
            time.sleep(3)  # Wait for dropdown to fully appear

            # Take a screenshot of the dropdown
            driver.save_screenshot("board_dropdown_open.png")
            files.download("board_dropdown_open.png")

            # Select the specific board using the data-test-id structure
            specific_board = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"[data-test-id='board-row-{board_name}']")))
            logger.info(f"Found specific board element: {board_name}")
            specific_board.click()

            logger.info(f"Selected board: {board_name}")
            time.sleep(2)
        except Exception as e:
            logger.error(f"Error selecting board with data-test-id: {str(e)}")
            # Fall back to the previous methods
            try:
                # Look for board by name in dropdown items
                board_options = driver.find_elements(By.CSS_SELECTOR, "div[role='option'] div.X8m")
                for option in board_options:
                    option_text = option.text.strip()
                    logger.info(f"Found board option: {option_text}")
                    if board_name.lower() in option_text.lower():
                        # Click the parent option element, not just the text
                        parent = driver.execute_script("return arguments[0].closest('[role=\"option\"]')", option)
                        if parent:
                            parent.click()
                            logger.info(f"Selected board: {option_text}")
                            time.sleep(2)
                            break
            except Exception as e:
                logger.error(f"Error selecting board by option text: {str(e)}")
                # Try JavaScript approach as last resort
                try:
                    driver.execute_script("""
                        // Find and click the board dropdown
                        var boardDropdown = document.querySelector('[data-test-id="board-dropdown-select-button"]');
                        if (boardDropdown) {
                            boardDropdown.click();

                            // Wait for dropdown to appear
                            setTimeout(function() {
                                // Find the board option by name
                                var boardName = arguments[0];
                                var options = Array.from(document.querySelectorAll('div[role="option"]'));

                                // Look for the board by name
                                for (var i = 0; i < options.length; i++) {
                                    if (options[i].innerText.toLowerCase().includes(boardName.toLowerCase())) {
                                        options[i].click();
                                        console.log("Selected board via JavaScript: " + options[i].innerText);
                                        break;
                                    }
                                }
                            }, 1000);
                        }
                    """, board_name)
                    logger.info("Attempted to select board via JavaScript")
                except Exception as e:
                    logger.error(f"Error selecting board via JavaScript: {str(e)}")

        # Take a screenshot before publishing
        driver.save_screenshot("before_publish.png")
        files.download("before_publish.png")

        # Find and click the Publish button
        logger.info("Publishing pin...")
        try:
            # Based on the screenshot, look for the red Publish button
            publish_button = driver.find_element(By.CSS_SELECTOR, "button.RCK.Hsu.USg.adn.gn8.L4E.kVc.CCY.oRi.lnZ.wsz")
            publish_button_text = publish_button.text.strip()
            logger.info(f"Found publish button: {publish_button_text}")
            if "publish" in publish_button_text.lower():
                publish_button.click()
                logger.info("Clicked publish button")
            else:
                # Try to find by text content
                buttons = driver.find_elements(By.TAG_NAME, "button")
                for button in buttons:
                    if "publish" in button.text.lower():
                        button.click()
                        logger.info(f"Clicked button with text: {button.text}")
                        break
        except Exception as e:
            logger.error(f"Error finding publish button: {str(e)}")
            # Try JavaScript approach
            try:
                driver.execute_script("""
                    // Find a button with "Publish" text
                    var buttons = Array.from(document.querySelectorAll('button'));
                    for (var i = 0; i < buttons.length; i++) {
                        if (buttons[i].innerText.toLowerCase().includes('publish')) {
                            console.log("Found publish button via JavaScript: " + buttons[i].innerText);
                            buttons[i].click();
                            return true;
                        }
                    }

                    // If no "Publish" button found, look for one with a distinctive appearance (e.g., red)
                    var redButtons = Array.from(document.querySelectorAll('button')).filter(function(btn) {
                        var style = window.getComputedStyle(btn);
                        var bgColor = style.backgroundColor.toLowerCase();
                        return bgColor.includes('rgb(230, 0, 35)') || bgColor.includes('#e60023');
                    });

                    if (redButtons.length > 0) {
                        console.log("Found red button, clicking it");
                        redButtons[0].click();
                        return true;
                    }

                    return false;
                """)
                logger.info("Attempted to click publish button via JavaScript")
            except Exception as e:
                logger.error(f"Error clicking publish button via JavaScript: {str(e)}")

        # Wait for the pin to be published
        time.sleep(15)

        # Check if we've been redirected (sign of success)
        current_url = driver.current_url
        logger.info(f"Current URL after publish attempt: {current_url}")

        # Take a final screenshot
        driver.save_screenshot("pin_published.png")
        files.download("pin_published.png")

        # Determine if publish was successful
        success = False
        if "pin-builder" not in current_url:
            success = True
            logger.info("Pin appears to have been published successfully (redirected from pin-builder)")
        else:
            # Check for success messages on the page
            success_text = driver.execute_script("""
                return document.body.innerText.includes('Saved to') ||
                       document.body.innerText.includes('Pin created') ||
                       document.body.innerText.includes('Your Pin was saved') ||
                       document.body.innerText.includes('Successfully published');
            """)
            if success_text:
                success = True
                logger.info("Pin appears to have been published successfully (success message found)")

        if success:
            logger.info("Pin published successfully!")
            return True
        else:
            logger.warning("Pin may not have been published successfully")
            return False

    except Exception as e:
        logger.error(f"Error during Pinterest upload: {str(e)}")
        # Include screenshot of the error state
        try:
            driver.save_screenshot("error_state.png")
            files.download("error_state.png")
        except:
            pass
        return False

    finally:
        # Close the browser
        logger.info("Closing browser...")
        try:
            driver.quit()
        except:
            pass

# Get user inputs
EMAIL = "mojomaniac2005@gmail.com"
PASSWORD = "Mojo@2005"
TITLE = "Sample Wallpaper Title"
DESCRIPTION = "Sample Wallpaper"
BOARD_NAME = "Wallpapers"
LINK_URL = "https://www.behance.net/shravanpandala2005"

# Upload the wallpaper
result = upload_to_pinterest(EMAIL, PASSWORD, image_path, TITLE, DESCRIPTION, BOARD_NAME, LINK_URL)

if result:
    print("Wallpaper uploaded successfully!")
else:
    print("Failed to upload wallpaper, check the logs for details.")

Please upload a wallpaper image:
